In [22]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [23]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [24]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [25]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [26]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [27]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [28]:
## 
n='Input_A6_011' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A6_011
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.0011873789203363802
test_rmse: 0.0011793076030212463
test_r2: 0.7485698201073624
feature number: 3
train_rmse: 0.001156164865557059
test_rmse: 0.001158559068342922
test_r2: 0.757030790208105
feature number: 4
train_rmse: 0.001150165002150566
test_rmse: 0.001153699065402764
test_r2: 0.7592450975210349
feature number: 5
train_rmse: 0.0011446746395264661
test_rmse: 0.0011517203628668956
test_r2: 0.7598274529850031
feature number: 6
train_rmse: 0.0011378939280024861
test_rmse: 0.001148425048570017
test_r2: 0.7612320983056858
feature number: 7
train_rmse: 0.0011348670130296502
test_rmse: 0.0011442370831728332
test_r2: 0.762888913734182
feature number: 8
train_rmse: 0.0011332746414664198
test_rmse: 0.001149764170609064
test_r2: 0.7604601645431159
feature number: 9
train_rmse: 0.0011328325377942899
test_rmse: 0.0011532501709575335
test_r2: 0.7589339494532092
feature number: 10
train_rmse: 0.00112933729967

train_rmse: 0.001137312630106381
test_rmse: 0.0011778394808019828
test_r2: 0.7490853280237132
feature number: 14
train_rmse: 0.0011317790847532004
test_rmse: 0.0011768504631234735
test_r2: 0.7493809711370277
feature number: 15
train_rmse: 0.0011303855538733472
test_rmse: 0.0011858153820706089
test_r2: 0.745445108839752
feature number: 16
train_rmse: 0.0011299504266745668
test_rmse: 0.0011873436765731777
test_r2: 0.7447641846340092
feature number: 17
train_rmse: 0.0011292995847708089
test_rmse: 0.001193106016024972
test_r2: 0.7423874910017478
feature number: 18
train_rmse: 0.0011276427641675475
test_rmse: 0.001192469620452429
test_r2: 0.7426600536741522
feature number: 19
train_rmse: 0.0011249496612165248
test_rmse: 0.0011979555502069396
test_r2: 0.7403105671536448
feature number: 20
train_rmse: 0.001121896060108658
test_rmse: 0.001203570189883246
test_r2: 0.7379546855670547
Test_rmse_min = 1.fea_num: 9  2.rmse: 0.0011671389603255472
Test_r2_max = 1. fea_num: 9  2.r2: 0.7538257768690437

train_rmse: 0.0011723504069246523
test_rmse: 0.0011861867385420171
test_r2: 0.7458813197221741
feature number: 5
train_rmse: 0.0011652388666346858
test_rmse: 0.0011857204007903644
test_r2: 0.7459923333037043
feature number: 6
train_rmse: 0.0011565469972676422
test_rmse: 0.0011786269951433903
test_r2: 0.7490100386525098
feature number: 7
train_rmse: 0.0011516376422223962
test_rmse: 0.0011705941005210625
test_r2: 0.7523255139166392
feature number: 8
train_rmse: 0.0011511895807999168
test_rmse: 0.0011718301575084143
test_r2: 0.7517609633088537
feature number: 9
train_rmse: 0.001145962117254555
test_rmse: 0.001167447420189441
test_r2: 0.7535253135110136
feature number: 10
train_rmse: 0.0011443739507484146
test_rmse: 0.0011737435418534017
test_r2: 0.7507842162413108
feature number: 11
train_rmse: 0.0011422237530820736
test_rmse: 0.0011711431957028894
test_r2: 0.7519193879332579
feature number: 12
train_rmse: 0.0011402866776380656
test_rmse: 0.0011714941478834712
test_r2: 0.751733674181059
f

# 可以修改第一層的神經元數量

In [29]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [30]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 6
test_rmse_ave: 0.003410044039591627
[0.004164544736453558, 0.008950920590390658, 0.0019352476291972017, 0.002003801494396044, 0.006957433499862122, 0.002663820320067068, 0.0012449037560184465, 0.0022668055920195824, 0.002686035695072308, 0.0012269270824392768]


test_r2_ave: -2.107952882529316
[-2.3879293981591005, -13.783212800717322, 0.4354110696241813, 0.3972987347805197, -6.649031456497085, -0.19583503983528905, 0.6207766156020327, 0.15284824868425917, -0.383333072487581, 0.7134782737122248]


feature number: 7
test_rmse_ave: 0.004044113121642446
[0.006502669860027458, 0.011004726746979998, 0.0013639168001342104, 0.0019871950051302493, 0.0019088035652138646, 0.003988746431576659, 0.006649670132142782, 0.0013510865574211579, 0.003461405729291835, 0.002222910388506244]


test_r2_ave: -3.90944403934945
[-7.260050431019641, -21.345596204282092, 0.719563183464408, 0.40724710380983964, 0.4242541795024515, -1.681231278977029, -9.819912761294772, 0.6990467703086582, -1.29

In [31]:
fea_num=[6,7,8,9]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 6
test_rmse_ave: 0.0037448419084706286
[0.003504940081660009, 0.001313149742496843, 0.009750406239610812, 0.0019027395132209125, 0.004488712661616236, 0.00668406467930975, 0.003962666055098161, 0.0028647419483309234, 0.0011826138881887408, 0.0017943842751738947]


test_r2_ave: -2.4382585326930704
[-1.3997195177217217, 0.681828023554915, -13.331930439437087, 0.45656027109595754, -2.1838450984202713, -6.529093792896589, -2.8423639997359995, -0.3530183207019144, 0.7318429925719687, 0.3871545547600379]


feature number: 7
test_rmse_ave: 0.0029988592871126197
[0.0027458643896274863, 0.001500392927537904, 0.002135774515047473, 0.003304569736816608, 0.0012749564681377621, 0.0013770818506206011, 0.0014247531926852724, 0.013770619109058024, 0.001031018527757148, 0.0014235621538379146]


test_r2_ave: -2.7141374969019285
[-0.47284640035477654, 0.5846220518403284, 0.3123458461199017, -0.6391645513072086, 0.7431384895137879, 0.6804190165699396, 0.5032900806827617, -30.26364382655680

In [ ]:
fea_num=[6,7,8,9]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 6


In [ ]:
fea_num=[6,7,8,9]# 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')